### Interest Rate Swap Pricer

Inputs:
a. Currency
b. Notional Schedule
c. Zero Rates *
d. Payment Freq
e. Daycount Fraction



Steps:
a. Take  

In [20]:
import pandas as pd
import numpy as np
import datetime 
import matplotlib.pyplot as plt
from datetime import date
import os
import warnings
warnings.filterwarnings("ignore")

from bs4 import BeautifulSoup
import requests
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import lxml
import plotly.graph_objects as go


&rarr; for chrome


In [2]:
# def PullData(url):
#     driver = webdriver.Chrome(executable_path=r"C:\Program Files (x86)\Google\Chrome\chromedriver.exe")
#     driver.get(url)
#     time.sleep(2)
#     scroll_pause_time = 1
#     screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
#     i = 1
#     while True:
#         driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))  
#         i += 1
#         time.sleep(scroll_pause_time)
#         scroll_height = driver.execute_script("return document.body.scrollHeight;")  
#         if (screen_height) * i > scroll_height:
#             break
#     while True:
#         try:
#             loadMoreButton = driver.find_element_by_class_name("tv-load-more__btn")
#             loadMoreButton.click()
#             time.sleep(5)
#         except:
#             break
#     zero_rates = pd.read_html(driver.page_source)[1]
#     driver.close()
#     zero_rates = zero_rates.iloc[:,[0,2,3]].replace('—',np.nan).dropna()
#     zero_rates.columns = ["Ticker","Maturity", "Yield"]
#     zero_rates["Maturity"] = pd.to_datetime(zero_rates["Maturity"])
#     zero_rates[["Yield"]] = zero_rates[["Yield"]].replace('−','-', regex=True)
#     zero_rates.iloc[:,0] = zero_rates.iloc[:,0].apply(lambda x: x[0:5])
#     zero_rates["Yield"] = pd.to_numeric(zero_rates["Yield"])
#     return zero_rates

 safaridriver --enable    

In [3]:
def PullData(url):
  
    driver = webdriver.Safari()
    driver.get(url)
    time.sleep(2) 

   
    scroll_pause_time = 1
    screen_height = driver.execute_script("return window.screen.height;")  # Get screen height
    i = 1

    while True:
        driver.execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        i += 1
        time.sleep(scroll_pause_time)
        scroll_height = driver.execute_script("return document.body.scrollHeight;")  
        if (screen_height) * i > scroll_height:
            break  # Stop scrolling when bottom is reached

    # Click "Load More" button (if available)
    while True:
        try:
            load_more_button = driver.find_element("class name", "tv-load-more__btn")
            load_more_button.click()
            time.sleep(5)  # Wait for content to load
        except NoSuchElementException:
            break  # No more "Load More" button

    # Extract table data

    tables = pd.read_html(driver.page_source)
    # for i, table in enumerate(tables):
    #     print(f"Table {i}:")
    #     print(table.head())  

    # zero_rates = pd.read_html(driver.page_source)[1]
    driver.quit()  # close the browser
    
    zero_rates = tables[1]
    print("Columns in extracted table:", zero_rates.columns)
    zero_rates.columns = ["Symbol", "Coupon", "Yield %", "Maturity date", "Term to maturity", "Price", "Change %", "Change"]
    zero_rates = zero_rates[["Symbol", "Maturity date", "Yield %"]]
    zero_rates.columns = ["Ticker", "Maturity", "Yield"]

    zero_rates["Maturity"] = pd.to_datetime(zero_rates["Maturity"], errors="coerce")

 
    zero_rates["Yield"] = zero_rates["Yield"].str.replace('%', '', regex=False) 
    zero_rates["Yield"] = pd.to_numeric(zero_rates["Yield"], errors="coerce")  


    zero_rates["Ticker"] = zero_rates["Ticker"].str[:5]
    return zero_rates


In [4]:
ticker_meta = pd.read_csv('ticker.csv')
url = "https://www.tradingview.com/markets/bonds/prices-all/"
zero_rates = PullData(url)
zero_rates.to_csv('zero_rates.csv', index=False)

Columns in extracted table: Index(['Symbol', 'Coupon', 'Yield %', 'Maturity date', 'Term to maturity',
       'Price', 'Change %', 'Change'],
      dtype='object')


In [5]:
zero_rates["T"] = zero_rates["Maturity"].apply(lambda x: ((x - datetime.datetime.now()).days)/365)

In [6]:
zero_rates["Df"] = 1/(1+zero_rates["Yield"]/100)**zero_rates["T"]

In [7]:
zero_rates = pd.merge(zero_rates, ticker_meta).dropna()

In [8]:
zero_rates

,Ticker,Maturity,Yield,T,Df,Country,Currency,Tenor
0,US01M,2025-03-25,4.321,0.082192,0.996529,UnitedStates,USD,0.833
1,US02M,2025-04-22,4.322,0.158904,0.993299,UnitedStates,USD,0.167
2,US03M,2025-05-22,4.307,0.241096,0.989885,UnitedStates,USD,0.250
3,US06M,2025-08-21,4.338,0.490411,0.979390,UnitedStates,USD,0.500
4,US01Y,2026-02-19,4.168,0.989041,0.960417,UnitedStates,USD,1.000
5,US02Y,2027-01-31,4.198,1.936986,0.923436,UnitedStates,USD,2.000
6,US03Y,2028-02-15,4.207,2.978082,0.884508,UnitedStates,USD,3.000
7,US05Y,2030-01-31,4.269,4.939726,0.813426,UnitedStates,USD,5.000
8,US07Y,2032-01-31,4.355,6.939726,0.743915,UnitedStates,USD,7.000
9,US10Y,2035-02-15,4.431,9.983562,0.648657,UnitedStates,USD,10.000


* T : The remaining time to maturity in years, calculated from today’s date.
* Df discount_factor : The present value of $1 received at maturity. Lower values indicate higher discounting (i.e., higher interest rates)
* yields are annualised
* Tenor: bond term in years

In [9]:
disc_factors = zero_rates.loc[:,["Currency", "T", "Df"]].reset_index(drop=True)

In [10]:
disc_factors

,Currency,T,Df
0,USD,0.082192,0.996529
1,USD,0.158904,0.993299
2,USD,0.241096,0.989885
3,USD,0.490411,0.979390
4,USD,0.989041,0.960417
5,USD,1.936986,0.923436
6,USD,2.978082,0.884508
7,USD,4.939726,0.813426
8,USD,6.939726,0.743915
9,USD,9.983562,0.648657


In [11]:
disc_factors[disc_factors.Currency == "USD"]

,Currency,T,Df
0,USD,0.082192,0.996529
1,USD,0.158904,0.993299
2,USD,0.241096,0.989885
3,USD,0.490411,0.979390
4,USD,0.989041,0.960417
5,USD,1.936986,0.923436
6,USD,2.978082,0.884508
7,USD,4.939726,0.813426
8,USD,6.939726,0.743915
9,USD,9.983562,0.648657


In [12]:
from utils import Instantaneous_Forward_Rate

In [13]:
StDate = datetime.date(2025, 3, 25) 
TrDate= datetime.date.today()
Tenor = 1.0  
curr = "USD" 
StDate
data= disc_factors


In [14]:
help(Instantaneous_Forward_Rate)

Help on function Instantaneous_Forward_Rate in module utils:

Instantaneous_Forward_Rate(StDate, Tenor, curr, data, TrDate=datetime.date(2025, 2, 22))
    interpolation to estimate discount factors at specific times



In [15]:
fwd_rate = Instantaneous_Forward_Rate(StDate, Tenor, curr, data)
print("Forward Rate:", fwd_rate)

Forward Rate: 0.041536804517859655


In [16]:
Instantaneous_Forward_Rate(datetime.date.today() + datetime.timedelta(days=1800), 0.25, "USD", data)

0.043234254706632384

In [18]:
spot = zero_rates[zero_rates["Currency"]==curr]["Yield"]
fwd3m, fwd6m, fwd1y = [], [], []
spottimelist = zero_rates[zero_rates["Currency"]==curr]["Tenor"]
timelist = np.linspace(0,20,80)
for i in timelist:
    fwd3m.append(Instantaneous_Forward_Rate(datetime.date.today() + datetime.timedelta(days=i*365), 0.25, "USD", data))
    fwd6m.append(Instantaneous_Forward_Rate(datetime.date.today() + datetime.timedelta(days=i*365), 0.5, "USD", data))
    fwd1y.append(Instantaneous_Forward_Rate(datetime.date.today() + datetime.timedelta(days=i*365), 1, "USD", data))

In [21]:
fig = go.Figure()


fig.add_trace(go.Scatter(x=spottimelist, y=spot, mode='lines', name='Spot Rate'))

fig.add_trace(go.Scatter(x=timelist, y=np.dot(fwd3m, 100), mode='lines', name='3M Fwd'))
fig.add_trace(go.Scatter(x=timelist, y=np.dot(fwd6m, 100), mode='lines', name='6M Fwd'))
fig.add_trace(go.Scatter(x=timelist, y=np.dot(fwd1y, 100), mode='lines', name='1Y Fwd'))

fig.update_layout(
    title="Forward Rate Curves",
    xaxis_title="Time to Maturity (Years)",
    yaxis_title="Forward Rate (%)",
    template="plotly_dark", 
    width=900,
    height=500
)

fig.show()

fine-tune the forward rate model(smothing the spike ?!) ...


In [29]:
curr = "USD"

In [30]:
zero_rates[zero_rates["Currency"]==curr]

,Ticker,Maturity,Yield,T,Df,Country,Currency,Tenor
0,US01M,2025-03-25,4.321,0.082192,0.996529,UnitedStates,USD,0.833
1,US02M,2025-04-22,4.322,0.158904,0.993299,UnitedStates,USD,0.167
2,US03M,2025-05-22,4.307,0.241096,0.989885,UnitedStates,USD,0.250
3,US06M,2025-08-21,4.338,0.490411,0.979390,UnitedStates,USD,0.500
4,US01Y,2026-02-19,4.168,0.989041,0.960417,UnitedStates,USD,1.000
5,US02Y,2027-01-31,4.198,1.936986,0.923436,UnitedStates,USD,2.000
6,US03Y,2028-02-15,4.207,2.978082,0.884508,UnitedStates,USD,3.000
7,US05Y,2030-01-31,4.269,4.939726,0.813426,UnitedStates,USD,5.000
8,US07Y,2032-01-31,4.355,6.939726,0.743915,UnitedStates,USD,7.000
9,US10Y,2035-02-15,4.431,9.983562,0.648657,UnitedStates,USD,10.000


In [22]:
zero_rates["T"][0] 

0.0821917808219178

In [23]:
datetime.date.today() + datetime.timedelta(days=5)

datetime.date(2025, 2, 27)

In [24]:
datetime.datetime.now() + datetime.timedelta(days=zero_rates["T"][0] *365)

datetime.datetime(2025, 3, 24, 18, 43, 44, 884447)

In [25]:
pd.read_csv('https://home.treasury.gov/resource-center/data-chart-center/interest-rates/daily-treasury-rates.csv/all/202202?type=daily_treasury_yield_curve&field_tdr_date_value_month=202202&page&_format=csv')


,Date,1 Mo,2 Mo,3 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr,20 Yr,30 Yr
0,02/28/2022,0.06,0.20,0.35,0.69,1.01,1.44,1.62,1.71,1.81,1.83,2.25,2.17
1,02/25/2022,0.03,0.20,0.33,0.71,1.13,1.55,1.76,1.86,1.96,1.97,2.37,2.29
2,02/24/2022,0.05,0.18,0.32,0.65,1.08,1.54,1.73,1.84,1.94,1.96,2.36,2.28
3,02/23/2022,0.02,0.20,0.34,0.74,1.16,1.58,1.78,1.89,1.98,1.99,2.36,2.29
4,02/22/2022,0.03,0.20,0.37,0.73,1.17,1.56,1.74,1.85,1.93,1.94,2.31,2.24
5,02/18/2022,0.03,0.19,0.35,0.65,1.03,1.47,1.68,1.82,1.90,1.92,2.30,2.24
6,02/17/2022,0.06,0.23,0.36,0.65,1.05,1.49,1.70,1.85,1.94,1.97,2.35,2.31
7,02/16/2022,0.03,0.17,0.38,0.67,1.09,1.52,1.75,1.90,2.00,2.03,2.39,2.34
8,02/15/2022,0.02,0.19,0.40,0.72,1.11,1.58,1.80,1.94,2.03,2.05,2.42,2.37
9,02/14/2022,0.03,0.25,0.43,0.76,1.13,1.58,1.80,1.90,1.98,1.98,2.35,2.29
